# Answer Key to Linear Regression Quiz

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, count, lit, udf, avg
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Description Length Features

In [5]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [7]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("DescLength", body_length(df.words))

In [8]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

In [16]:
number_of_tags = udf(lambda x: len(x.split(" ")), IntegerType())
df = df.withColumn("NumTags", number_of_tags(df.Tags))

# Question
Build a linear regression model using the length of the combined Title + Body fields. What is the value of r^2 when fitting a model with maxIter=5, regParam=0.0, fitIntercept=False, solver="normal"?

In [17]:
lr = LinearRegression(maxIter=5, regParam=0.0, fitIntercept=False, \
                      solver="normal")

In [18]:
df.groupby("NumTags").agg(avg(col("DescLength"))).orderBy("NumTags").show()

+-------+------------------+
|NumTags|   avg(DescLength)|
+-------+------------------+
|      1|143.68776158175783|
|      2| 162.1539186134137|
|      3|181.26021064340088|
|      4|201.46530249110322|
|      5|227.64375266524522|
+-------+------------------+



In [19]:
data = df.select(col("NumTags").alias("label"), col("DescVec").alias("features"))
data.head()

Row(label=5, features=DenseVector([96.0]))

In [20]:
lrModel_q1 = lr.fit(data)

In [21]:
lrModel_q1.summary.r2

0.4455149596308462